## Faceta

*O fecetin* envolve a subdivisão de um conjunto de dados em grupos e a criação de um gráfico separado para cada grupo. Em notebooks anteriores, aprendemos como criar gráficos facetados usando os métodos `linha` e `coluna`. Vamos primeiro rever esses canais e depois mostrar como eles são instâncias do operador `facet` mais geral.

Vamos começar com um histograma básico de valores de temperatura máxima em Seattle:

In [1]:
alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q')
)

NameError: name 'alt' is not defined

_Como é que este perfil de temperatura muda com base no tempo de um determinado dia - isto é, se houve chuvisco, nevoeiro, chuva, neve ou sol?_

Vamos usar o método `column` para facetar os dados por tipo de clima.Também podemos usar `color` como uma codificação redundante, usando um intervalo de cores personalizado:

In [ ]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(weather).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors),
  alt.Column('weather:N')
).properties(
  width=150,
  height=150
)

Sem surpresa, os raros dias de neve concentram-se nas temperaturas mais frias, seguidos de dias de chuva e nevoeiro. Os dias de sol são mais quentes e, apesar dos estereótipos de Seattle, são os mais abundantes.No entanto, como qualquer habitante de Seattle lhe pode dizer, o chuvisco aparece ocasionalmente, independentemente da temperatura!_

Além dos métodos `row` e `column` *dentro* de uma definição de gráfico, podemos pegar uma definição básica de gráfico e aplicar facetes usando um operador `facet` explícito.

Vamos recriar o gráfico acima, mas desta vez usando `facet`.Começamos com a mesma definição básica de histograma, mas removemos a fonte de dados, a transformação do filtro e o canal da coluna. Podemos então invocar o método `facet`, passando os dados e especificando que devemos facetar as colunas de acordo com o campo `weather`. O método `facet` aceita ambos os argumentos `row` e `column`. Os dois podem ser usados juntos para criar uma grade 2D de gráficos facetados.

Finalmente, incluímos nossa transformação de filtro, aplicando-a ao gráfico facetado de nível superior. Embora pudéssemos aplicar a transformação do filtro à definição do histograma como antes, isso é um pouco menos eficiente. Em vez de filtrar os valores de “Nova York” dentro de cada célula de faceta, a aplicação do filtro ao gráfico facetado permite que o Vega-Lite saiba que podemos filtrar esses valores antecipadamente, antes da subdivisão da faceta.

In [ ]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart().mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors)
).properties(
  width=150,
  height=150
).facet(
  data=weather,
  column='weather:N'
).transform_filter(
  'datum.location == "Seattle"'
)

Dado todo o código extra acima, porque é que queremos usar um operador `facet` explícito? Para gráficos básicos, nós certamente devemos usar os métodos `column` ou `row` se pudermos. No entanto, usar o operador `facet` explicitamente é útil se quisermos facetar visualizações compostas, como gráficos em camadas.

Vamos revisitar nossos gráficos de temperatura em camadas de antes. Em vez de plotar os dados de Nova York e Seattle no mesmo gráfico, vamos dividi-los em facetas separadas. As definições dos gráficos individuais são praticamente as mesmas de antes: um gráfico de área e um gráfico de linhas. A única diferença é que, desta vez, não passaremos os dados diretamente para os construtores de gráficos; esperaremos e os passaremos para o operador de faceta mais tarde. Nós podemos colocar os gráficos em camadas como antes, e então invocar `facet` no objeto gráfico em camadas, passando os dados e especificando as facetas `column` baseadas no campo `location`:

In [ ]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
)

Os gráficos facetados que vimos até agora utilizam os mesmos domínios de escala de eixo nas células facetadas.Esta predefinição de utilização de escalas e eixos *compartilhados* ajuda a uma comparação precisa dos valores.No entanto, em alguns casos, pode desejar escalar cada gráfico independentemente, por exemplo, se o intervalo de valores nas células for significativamente diferente.
À semelhança dos gráficos em camadas, os gráficos facetados também suportam a _resolução_ para escalas ou eixos independentes entre gráficos.Vamos ver o que acontece se chamarmos o método `resolver_eixo` para solicitar eixos y `independentes`:

In [ ]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura méida (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_axis(y='independent')


_O gráfico acima parece praticamente inalterado, mas o gráfico de Seattle agora inclui seu próprio eixo.

E se ao invés disso nós chamarmos `resolve_scale` para resolver os domínios de escala subjacentes?

In [ ]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média(°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=weather,
  column='location:N'
).resolve_scale(y='independent')

Agora vemos células de faceta com domínios de escala de eixos diferentes. Neste caso, usar escalas independentes parece ser uma má idéia! Os domínios não são muito diferentes e poderíamos ser levados a pensar que Nova Iorque e Seattle têm temperaturas máximas de verão semelhantes.

Para usar um cliché: só porque *pode* fazer algo, não significa que *deva*...